# Chapter4 Select

本文将使用 rexon_metals.db 数据库，其中包含了 CUSTOMER，CUSTOMER_ORDER 以及 PRODUCT 三张表。

In [1]:
%load_ext sql
%sql sqlite:///DataBase/rexon_metals.db

'Connected: @DataBase/rexon_metals.db'

使用 * 选取 CUSTOMER 表中的所有列：

In [2]:
%sql select * from CUSTOMER;

 * sqlite:///DataBase/rexon_metals.db
Done.


CUSTOMER_ID,NAME,REGION,STREET_ADDRESS,CITY,STATE,ZIP
1,LITE Industrial,Southwest,729 Ravine Way,Irving,TX,75014
2,Rex Tooling Inc,Southwest,6129 Collie Blvd,Dallas,TX,75201
3,Re-Barre Construction,Southwest,9043 Windy Dr,Irving,TX,75032
4,Prairie Construction,Southwest,264 Long Rd,Moore,OK,62104
5,Marsh Lane Metal Works,Southeast,9143 Marsh Ln,Avondale,LA,79782


也可以指定选取某几列，如 CUSTOMER_ID, NAME

In [7]:
%sql select CUSTOMER_ID, NAME from CUSTOMER;

 * sqlite:///DataBase/rexon_metals.db
Done.


CUSTOMER_ID,NAME
1,LITE Industrial
2,Rex Tooling Inc
3,Re-Barre Construction
4,Prairie Construction
5,Marsh Lane Metal Works


## alias

SELECT 不经可以简单的选取某几列数据，还可以对它们进行运算，如利用 PRODUCT 表中的 PRICE 计算相应的税后价格 TAXED_PRICE：

In [11]:
%%sql
select
PRODUCT_ID,
DESCRIPTION,
PRICE,
PRICE * 1.07 as TAXED_PRICE
from PRODUCT;

 * sqlite:///DataBase/rexon_metals.db
Done.


PRODUCT_ID,DESCRIPTION,PRICE,TAXED_PRICE
1,Copper,7.51,8.0357
2,Aluminum,2.58,2.7606
3,Silver,15,16.05
4,Steel,12.31,13.171700000000001
5,Bronze,4,4.28
6,Duralumin,7.6,8.132
7,Solder,14.16,15.151200000000001
8,Stellite,13.31,14.241700000000002
9,Brass,4.75,5.0825000000000005


- 当在 SQL 中对任何东西赋予变量名时（不论是alias, a column name, a table name 或者其他 entity），都应该使用下划线 (_) 作为占位符，如果使用空格的话，将会出现错误。


我们可以使用 **别名**（alias）对表达式赋予名字，也可以在查询中用 alias 做为已存在的列的列名。比如将 UNTAXED_PRICE 作为 PRICE 的 alias，但这并不会改变表中的列名，而是在执行 SELECT 语句时赋予了一个新的名字：

## 内置函数

In [14]:
%%sql
select
PRODUCT_ID,
DESCRIPTION,
PRICE AS UNTAXED_PRICE,
PRICE * 1.07 as TAXED_PRICE
from PRODUCT;

 * sqlite:///DataBase/rexon_metals.db
Done.


PRODUCT_ID,DESCRIPTION,UNTAXED_PRICE,TAXED_PRICE
1,Copper,7.51,8.0357
2,Aluminum,2.58,2.7606
3,Silver,15,16.05
4,Steel,12.31,13.171700000000001
5,Bronze,4,4.28
6,Duralumin,7.6,8.132
7,Solder,14.16,15.151200000000001
8,Stellite,13.31,14.241700000000002
9,Brass,4.75,5.0825000000000005


每一个数据库平台都有内置函数（built-in-functions），SQLite 提供了 round() 函数来对数据进行四舍五入，如将 TAXED_PRICE 保留两位小数：

In [16]:
%%sql
select
PRODUCT_ID,
DESCRIPTION,
PRICE AS UNTAXED_PRICE,
round(PRICE * 1.07, 2) as TAXED_PRICE
from PRODUCT;

 * sqlite:///DataBase/rexon_metals.db
Done.


PRODUCT_ID,DESCRIPTION,UNTAXED_PRICE,TAXED_PRICE
1,Copper,7.51,8.04
2,Aluminum,2.58,2.76
3,Silver,15,16.05
4,Steel,12.31,13.17
5,Bronze,4,4.28
6,Duralumin,7.6,8.13
7,Solder,14.16,15.15
8,Stellite,13.31,14.24
9,Brass,4.75,5.08


<img style="float:center" src="https://x1a-alioss.oss-cn-shenzhen.aliyuncs.com/Screen Shot 2020-02-09 at 16.58.36.png" width="520" >

<center> 图1 SQLite 内置函数 </center>

## 文字拼接

表达式不一定只对数字类型的数据起作用，也可以用与文字型（text）或者其他类型的数据。对于文字型数据，一个有用的命令就是 *concatenation*（联合），可以将两段数据联合到一起。

在 SQLite 中，这个联合操作符为 (||)，可以将两边的数据联合到一起；举个例子，可以将 CUSTOMER 表中的 CITY 和 STATE 联合为一个 LOCAYION：

- Concatenation 对任何的数据类型都适用（numbers, dates,...）
- 很多数据库平台使用 (||) 来进行拼接，但 MySQL 使用 CONCAT（） 函数

In [18]:
%%sql
select 
NAME,
CITY ||','|| STATE as LOCATION
from CUSTOMER;

 * sqlite:///DataBase/rexon_metals.db
Done.


NAME,LOCATION
LITE Industrial,"Irving,TX"
Rex Tooling Inc,"Dallas,TX"
Re-Barre Construction,"Irving,TX"
Prairie Construction,"Moore,OK"
Marsh Lane Metal Works,"Avondale,LA"


也可以将几个 fields 合成为一个单独的 SHIP_ADDRESS （邮件地址）：

In [20]:
%%sql
select
NAME,
STREET_ADDRESS || ' ' || CITY || ',' || STATE || ' ' || ZIP as SHIP_ADDRESS
from CUSTOMER;

 * sqlite:///DataBase/rexon_metals.db
Done.


NAME,SHIP_ADDRESS
LITE Industrial,"729 Ravine Way Irving,TX 75014"
Rex Tooling Inc,"6129 Collie Blvd Dallas,TX 75201"
Re-Barre Construction,"9043 Windy Dr Irving,TX 75032"
Prairie Construction,"264 Long Rd Moore,OK 62104"
Marsh Lane Metal Works,"9143 Marsh Ln Avondale,LA 79782"
